## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = path.join(getcwd(), "Sorting-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Workloads\\Sorting\\Sorting-S'

In [6]:
atprogram(project_path, verbose=2)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: Sorting-S.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe" -o Sorting-S.elf  a

0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
if live_plot:
    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
        "read_mode": [True, True, True, True],
        "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
        "plot_pins": [False, False, True, True],
        "gpio_delay_time" : 0.0007,
        "plot_pins_method": "line",
        "plot_xmax": 1,
        "fig": fig,
        "window_title": "Experiment Sorting",
        "file_name_base": "experiment_sorting"
    }
else:
    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
        "read_mode": [True, True, True, True],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_sorting"
    }

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [11]:
def stop_function(pin_values):
    return all(pin_values)

In [12]:
repetitions = 10

Perform the measurement.

In [14]:
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
        
charges = []
times = []

with DGILibExtra(**config_dict) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(2,stop_fn)
        charge, time = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        charges += charge
        times += time
        if len(charge) > 1 or len(time) > 1:
            print(f"Parsing of measurement data failed, charge:{charge}, time:{time}")
            break
        dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {charge[0]*1e3:.06} mC, time: {time[0]:.06} s")
        dgilib.empty_data()
    
    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/repetitions*1e3:.06} mC, time: {sum(times)/repetitions:.06} s")

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  ax.set_xlim((valmin, valmax))


In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [17]:
df

,times,charges
0,0.059003,0.00002
1,0.059006,0.00002
2,0.059074,0.00002
3,0.059076,0.00002
4,0.059070,0.00002
5,0.059009,0.00002
6,0.059075,0.00002
7,0.059011,0.00002
8,0.059003,0.00002
9,0.059074,0.00002


In [18]:
df.describe()

,times,charges
count,10.000000,1.000000e+01
mean,0.059040,1.984910e-05
std,0.000036,2.654677e-08
min,0.059003,1.981294e-05
25%,0.059007,1.983030e-05
50%,0.059040,1.984340e-05
75%,0.059074,1.987031e-05
max,0.059076,1.989624e-05


In [21]:
sum(charges)/repetitions

1.984909516280826e-05

In [24]:
print("%.0100f" % (sum(charges)/repetitions))

0.0000198490951628082594449236503164613054650544654577970504760742187500000000000000000000000000000000
